# Dependencies and modules

In [32]:
# !pip3 install pandas_profiling --upgrade
# !pip install sweetviz

In [33]:
import numpy as np
import pandas as pd

# from pandas_profiling import ProfileReport
# import sweetviz as sv

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  roc_auc_score

from imblearn.under_sampling import EditedNearestNeighbours

# Initial EDA using Pandas-Profiling and SweetViz visualization tools

In [34]:
dataset = pd.read_csv("/content/drive/MyDrive/aug_train.csv")
# profile_v1 = ProfileReport(dataset, title="HR Analytics - Before Preprocessing", explorative=True)
# profile_v1.to_file("EDA_v1.html")

# analyze_report = sv.analyze(dataset)
# analyze_report.show_html('EDA-v1-SweetViz.html')

# Data Preprocessing

## **Removing unique column**: "enrolled_id"

In [35]:
dataset.drop(columns=['enrollee_id'],inplace=True)

## **Encoding columns using LabelEncoder**

In [36]:
dataset = dataset.apply(lambda series: pd.Series(
    LabelEncoder().fit_transform(series[series.notnull()]),
    index=series[series.notnull()].index
))

In [37]:
dataset.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,5,85,1.0,0,2.0,0.0,5.0,21.0,NaN,NaN,0.0,35,1
1,77,48,1.0,1,2.0,0.0,5.0,6.0,4.0,5.0,4.0,46,0
2,64,14,NaN,1,0.0,0.0,5.0,15.0,NaN,NaN,5.0,82,0
3,14,52,NaN,1,NaN,0.0,1.0,20.0,NaN,5.0,5.0,51,1
4,50,45,1.0,0,2.0,2.0,5.0,21.0,4.0,1.0,3.0,7,0


## Splitting train and test dataset

In [38]:
X=dataset.drop(columns=['target'],axis=1)
y=dataset['target']
x_train,x_test,y_train,y_test = train_test_split(X,y, train_size=0.8, stratify=y, random_state=42)

## Forward Fill Missing Values


In [39]:
x_train['gender'] = x_train.sort_values(by=['city_development_index'])['gender'].fillna(method='ffill')
x_train['enrolled_university'] = x_train.sort_values(by=['relevent_experience'])['enrolled_university'].fillna(method='ffill')
x_train['major_discipline'] = x_train.sort_values(by=['education_level'])['major_discipline'].fillna(method='ffill')
x_train['education_level'] = x_train.sort_values(by=['major_discipline'])['education_level'].fillna(method='ffill')
x_train['experience'] = x_train.sort_values(by=['relevent_experience'])['experience'].fillna(method='ffill')
x_train['company_type'] = x_train.sort_values(by=['relevent_experience','company_size'])['company_type'].fillna(method='ffill')
x_train['company_size'] = x_train.sort_values(by=['relevent_experience','company_type'])['company_size'].fillna(method='ffill')
x_train['last_new_job'] = x_train.sort_values(by=['relevent_experience'])['last_new_job'].fillna(method='ffill')
x_train


,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
17855,64,14,1.0,0,2.0,0.0,5.0,1.0,4.0,5.0,0.0,89
17664,5,85,1.0,1,2.0,4.0,5.0,15.0,5.0,5.0,5.0,14
13404,85,77,1.0,0,2.0,0.0,5.0,3.0,2.0,2.0,4.0,35
13366,5,85,1.0,0,2.0,0.0,5.0,15.0,1.0,1.0,0.0,52
15670,95,67,0.0,0,0.0,0.0,5.0,15.0,4.0,5.0,0.0,154
...,...,...,...,...,...,...,...,...,...,...,...,...
10398,95,67,1.0,0,2.0,0.0,5.0,4.0,4.0,5.0,0.0,92
859,5,85,0.0,0,2.0,2.0,5.0,1.0,4.0,5.0,0.0,15
10566,74,75,0.0,0,2.0,2.0,5.0,5.0,1.0,5.0,0.0,33
3085,64,14,0.0,0,2.0,0.0,5.0,6.0,0.0,1.0,1.0,110


In [40]:
x_test['gender'] = x_test.sort_values(by=['major_discipline'])['gender'].fillna(method='ffill')
x_test['enrolled_university'] = x_test.sort_values(by=['relevent_experience'])['enrolled_university'].fillna(method='ffill')
x_test['major_discipline'] = x_test.sort_values(by=['education_level'])['major_discipline'].fillna(method='ffill')
x_test['education_level'] = x_test.sort_values(by=['major_discipline'])['education_level'].fillna(method='ffill')
x_test['experience'] = x_test.sort_values(by=['relevent_experience'])['experience'].fillna(method='ffill')
x_test['company_type'] = x_test.sort_values(by=['relevent_experience','company_size'])['company_type'].fillna(method='ffill')
x_test['company_type'] = x_test.sort_values(by=['relevent_experience','company_size'])['company_type'].fillna(method='bfill')
x_test['company_size'] = x_test.sort_values(by=['relevent_experience','company_type'])['company_size'].fillna(method='ffill')
x_test['last_new_job'] = x_test.sort_values(by=['relevent_experience'])['last_new_job'].fillna(method='ffill')
x_test

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
686,103,91,1.0,0,2.0,0.0,5.0,3.0,2.0,5.0,0.0,41
1106,116,27,0.0,1,0.0,0.0,5.0,11.0,5.0,5.0,5.0,17
8966,103,91,1.0,0,2.0,1.0,5.0,15.0,7.0,5.0,0.0,21
7961,38,63,0.0,0,2.0,0.0,5.0,1.0,2.0,4.0,0.0,77
5182,64,14,1.0,0,2.0,0.0,5.0,21.0,7.0,5.0,4.0,77
...,...,...,...,...,...,...,...,...,...,...,...,...
9055,5,85,1.0,1,2.0,0.0,5.0,13.0,2.0,5.0,0.0,171
11207,93,21,1.0,0,0.0,0.0,5.0,21.0,3.0,5.0,0.0,7
16790,5,85,1.0,1,2.0,0.0,5.0,6.0,5.0,4.0,2.0,217
5575,70,91,1.0,1,0.0,1.0,5.0,11.0,3.0,5.0,5.0,180


## Balancing training set using EditedNearestNeighbors

In [41]:
enn = EditedNearestNeighbours(n_neighbors=5,)
x_train, y_train = enn.fit_resample(x_train, y_train)

# EDA After Data Preprocessing

In [42]:
# report_comp=sv.compare([x_train,'Train'],[x_test,'Test']) 
# report_comp.show_html('EDA_Comparing.html')

# profile_v1 = ProfileReport(x_train, title="HR Analytics - After Preprocessing", explorative=True)
# profile_v1.to_file("EDA_Train.html")

# profile_v2 = ProfileReport(x_test, title="HR Analytics - After Preprocessing", explorative=True)
# profile_v2.to_file("EDA_Test.html")

# Setups of features (Feature Selection)

In [43]:
first_setup = ['city_development_index', 'experience','company_size','last_new_job','training_hours'] #Only numeric features
second_setup = ['city','gender','relevent_experience','enrolled_university','education_level','major_discipline','company_type'] #Only categorical features

#Models

## KNN Classification

### First Setup of Features

In [44]:
model = KNeighborsClassifier()
model.fit(x_train[first_setup], y_train)

pred = model.predict(x_test[first_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[first_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.6998720671438217 

              precision    recall  f1-score   support

           0       0.85      0.61      0.71      2877
           1       0.37      0.68      0.48       955

    accuracy                           0.63      3832
   macro avg       0.61      0.64      0.59      3832
weighted avg       0.73      0.63      0.65      3832



### Second Setup of Features

In [45]:
model = KNeighborsClassifier()
model.fit(x_train[second_setup], y_train)

pred = model.predict(x_test[second_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[second_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.715133019233604 

              precision    recall  f1-score   support

           0       0.85      0.79      0.82      2877
           1       0.47      0.57      0.52       955

    accuracy                           0.74      3832
   macro avg       0.66      0.68      0.67      3832
weighted avg       0.75      0.74      0.74      3832



### All Features

In [46]:
model = KNeighborsClassifier()
model.fit(x_train, y_train)

pred = model.predict(x_test)
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test)[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.6967561832697309 

              precision    recall  f1-score   support

           0       0.85      0.60      0.71      2877
           1       0.37      0.69      0.48       955

    accuracy                           0.63      3832
   macro avg       0.61      0.65      0.59      3832
weighted avg       0.73      0.63      0.65      3832



## Naive Bayes Classification

### First Setup of Features

In [47]:
model = GaussianNB()
model.fit(x_train[first_setup], y_train)

pred = model.predict(x_test[first_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[first_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.695296875199042 

              precision    recall  f1-score   support

           0       0.85      0.75      0.79      2877
           1       0.44      0.59      0.50       955

    accuracy                           0.71      3832
   macro avg       0.64      0.67      0.65      3832
weighted avg       0.74      0.71      0.72      3832



### Second Setup of Features

In [48]:
model = GaussianNB()
model.fit(x_train[second_setup], y_train)

pred = model.predict(x_test[second_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[second_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.6310274482399678 

              precision    recall  f1-score   support

           0       0.80      0.71      0.75      2877
           1       0.35      0.46      0.40       955

    accuracy                           0.65      3832
   macro avg       0.58      0.59      0.58      3832
weighted avg       0.69      0.65      0.67      3832



### All Features

In [49]:
model = GaussianNB()
model.fit(x_train, y_train)

pred = model.predict(x_test)
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test)[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7342577983537972 

              precision    recall  f1-score   support

           0       0.87      0.66      0.75      2877
           1       0.41      0.71      0.52       955

    accuracy                           0.68      3832
   macro avg       0.64      0.69      0.64      3832
weighted avg       0.76      0.68      0.70      3832



## Logistic Regression 

### First Setup Of Features

In [50]:
model = LogisticRegression(solver = 'newton-cg')
model.fit(x_train[first_setup], y_train)

pred = model.predict(x_test[first_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[first_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.6939509414802724 

              precision    recall  f1-score   support

           0       0.85      0.72      0.78      2877
           1       0.42      0.60      0.49       955

    accuracy                           0.69      3832
   macro avg       0.63      0.66      0.64      3832
weighted avg       0.74      0.69      0.71      3832



### Second Setup of Features

In [51]:
model = LogisticRegression(solver = 'newton-cg')
model.fit(x_train[second_setup], y_train)

pred = model.predict(x_test[second_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[second_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.6223349293093627 

              precision    recall  f1-score   support

           0       0.80      0.72      0.76      2877
           1       0.36      0.46      0.40       955

    accuracy                           0.66      3832
   macro avg       0.58      0.59      0.58      3832
weighted avg       0.69      0.66      0.67      3832



### All Features

In [52]:
model = LogisticRegression(solver = 'newton-cg')
model.fit(x_train, y_train)

pred = model.predict(x_test)
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test)[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7442893357136487 

              precision    recall  f1-score   support

           0       0.87      0.70      0.77      2877
           1       0.42      0.67      0.52       955

    accuracy                           0.69      3832
   macro avg       0.65      0.69      0.65      3832
weighted avg       0.76      0.69      0.71      3832



## RandomForest Classification

### First Setup Of Features

In [53]:
model = RandomForestClassifier(random_state=42)
model.fit(x_train[first_setup], y_train)

pred = model.predict(x_test[first_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[first_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7232055278640672 

              precision    recall  f1-score   support

           0       0.86      0.64      0.73      2877
           1       0.39      0.68      0.49       955

    accuracy                           0.65      3832
   macro avg       0.62      0.66      0.61      3832
weighted avg       0.74      0.65      0.67      3832



### Second Setup Of Features

In [54]:
model = RandomForestClassifier(random_state=42)
model.fit(x_train[second_setup], y_train)

pred = model.predict(x_test[second_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[second_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7483509764206825 

              precision    recall  f1-score   support

           0       0.87      0.69      0.77      2877
           1       0.43      0.70      0.53       955

    accuracy                           0.69      3832
   macro avg       0.65      0.69      0.65      3832
weighted avg       0.76      0.69      0.71      3832



### All Features

In [55]:
model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

pred = model.predict(x_test)
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test)[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7610887577410297 

              precision    recall  f1-score   support

           0       0.88      0.67      0.76      2877
           1       0.42      0.73      0.53       955

    accuracy                           0.68      3832
   macro avg       0.65      0.70      0.65      3832
weighted avg       0.77      0.68      0.70      3832



## SVM Classification

### First Setup Of Features

In [56]:
model = SVC(probability=True)
model.fit(x_train[first_setup], y_train)

pred = model.predict(x_test[first_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[first_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.6936790614132303 

              precision    recall  f1-score   support

           0       0.84      0.81      0.83      2877
           1       0.49      0.55      0.52       955

    accuracy                           0.74      3832
   macro avg       0.67      0.68      0.67      3832
weighted avg       0.76      0.74      0.75      3832



### Second Setup Of Features

In [57]:
model = SVC(probability=True)
model.fit(x_train[second_setup], y_train)

pred = model.predict(x_test[second_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[second_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.6531232541168721 

              precision    recall  f1-score   support

           0       0.81      0.76      0.79      2877
           1       0.39      0.46      0.43       955

    accuracy                           0.69      3832
   macro avg       0.60      0.61      0.61      3832
weighted avg       0.71      0.69      0.70      3832



### All Features

In [58]:
model = SVC(probability=True)
model.fit(x_train, y_train)

pred = model.predict(x_test)
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test)[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7360481304150811 

              precision    recall  f1-score   support

           0       0.84      0.80      0.82      2877
           1       0.48      0.55      0.51       955

    accuracy                           0.74      3832
   macro avg       0.66      0.68      0.67      3832
weighted avg       0.75      0.74      0.75      3832



## Neural Networks Classification

### First Setup of Features

In [59]:
model = MLPClassifier()
model.fit(x_train[first_setup], y_train)

pred = model.predict(x_test[first_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[first_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7028137949107108 

              precision    recall  f1-score   support

           0       0.85      0.72      0.78      2877
           1       0.42      0.61      0.49       955

    accuracy                           0.69      3832
   macro avg       0.63      0.66      0.63      3832
weighted avg       0.74      0.69      0.71      3832



### Second Setup of Features

In [60]:
model = MLPClassifier()
model.fit(x_train[second_setup], y_train)

pred = model.predict(x_test[second_setup])
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[second_setup])[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7072044578140042 

              precision    recall  f1-score   support

           0       0.84      0.75      0.79      2877
           1       0.43      0.59      0.50       955

    accuracy                           0.71      3832
   macro avg       0.64      0.67      0.65      3832
weighted avg       0.74      0.71      0.72      3832



### All Features

In [61]:
model = MLPClassifier()
model.fit(x_train, y_train)

pred = model.predict(x_test)
print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test)[:,1]),"\n")
print(classification_report(y_test,pred))

ROC AUC SCORE:  0.7593512002576855 

              precision    recall  f1-score   support

           0       0.88      0.69      0.77      2877
           1       0.43      0.71      0.54       955

    accuracy                           0.70      3832
   macro avg       0.66      0.70      0.66      3832
weighted avg       0.77      0.70      0.72      3832



# Hypertuning the top 3 models

## RandomForest Model

### All Features **(ROC AUC SCORE: 0.76)**

In [62]:
for n_estimators in [100,1000,5000]:
  for criterion in ['gini','entropy']:
    for max_depth in [10,15,30]:
      model = RandomForestClassifier(n_estimators = n_estimators, criterion= criterion, max_depth = max_depth, random_state=42)
      model.fit(x_train, y_train)
      pred = model.predict(x_test)
      print("{n_estimators: ",n_estimators,", criterion: ",criterion,", max_depth: ",max_depth,"}\n")
      print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test)[:,1]),"\n\n")
      print(classification_report(y_test,pred),"\n\n\n\n")

{n_estimators:  100 , criterion:  gini , max_depth:  10 }

ROC AUC SCORE:  0.7682213329402536 


              precision    recall  f1-score   support

           0       0.88      0.73      0.80      2877
           1       0.46      0.69      0.55       955

    accuracy                           0.72      3832
   macro avg       0.67      0.71      0.67      3832
weighted avg       0.77      0.72      0.73      3832
 




{n_estimators:  100 , criterion:  gini , max_depth:  15 }

ROC AUC SCORE:  0.7665012456620207 


              precision    recall  f1-score   support

           0       0.88      0.70      0.78      2877
           1       0.44      0.72      0.55       955

    accuracy                           0.70      3832
   macro avg       0.66      0.71      0.66      3832
weighted avg       0.77      0.70      0.72      3832
 




{n_estimators:  100 , criterion:  gini , max_depth:  30 }

ROC AUC SCORE:  0.7594012451160769 


              precision    recall  f1-score  

### Second Setup of Features **(ROC AUC SCORE: 0.748)**

In [63]:
for n_estimators in [100,1000,5000]:
  for criterion in ['gini','entropy']:
    for max_depth in [10,15,30]:
      model = RandomForestClassifier(n_estimators = n_estimators, criterion= criterion, max_depth = max_depth, random_state=42)
      model.fit(x_train[second_setup], y_train)
      pred = model.predict(x_test[second_setup])
      print("{n_estimators: ",n_estimators,", criterion: ",criterion,", max_depth: ",max_depth,"}\n")
      print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test[second_setup])[:,1]),"\n\n")
      print(classification_report(y_test,pred),"\n\n\n\n")

{n_estimators:  100 , criterion:  gini , max_depth:  10 }

ROC AUC SCORE:  0.7475109507249226 


              precision    recall  f1-score   support

           0       0.87      0.74      0.80      2877
           1       0.45      0.65      0.54       955

    accuracy                           0.72      3832
   macro avg       0.66      0.70      0.67      3832
weighted avg       0.76      0.72      0.73      3832
 




{n_estimators:  100 , criterion:  gini , max_depth:  15 }

ROC AUC SCORE:  0.7485364153686851 


              precision    recall  f1-score   support

           0       0.87      0.70      0.78      2877
           1       0.43      0.67      0.53       955

    accuracy                           0.70      3832
   macro avg       0.65      0.69      0.65      3832
weighted avg       0.76      0.70      0.71      3832
 




{n_estimators:  100 , criterion:  gini , max_depth:  30 }

ROC AUC SCORE:  0.7483509764206825 


              precision    recall  f1-score  

## Neural Network Model **(ROC AUC SCORE: 0.756)**

In [64]:
for hidden_layer_sizes in [(128),(8,16),(16,32,64),(32,32,64,128)]:
  for activation in ['logistic','tanh','relu']:
    for solver in ['lbfgs', 'sgd', 'adam']:
      for batch_size in [15, 20, 25]:
        model = MLPClassifier(hidden_layer_sizes = hidden_layer_sizes, activation = activation, solver = solver, batch_size=batch_size, max_iter=1000)
        model.fit(x_train, y_train)
        pred = model.predict(x_test)
        print("{hidden_layer_sizes: ",hidden_layer_sizes,", activation: ",activation,", solver: ",solver,", batch_size: ",batch_size,"}\n")
        print("ROC AUC SCORE: ", roc_auc_score(y_test, model.predict_proba(x_test)[:,1]),"\n\n")
        print(classification_report(y_test,pred),"\n\n\n\n")

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


{hidden_layer_sizes:  128 , activation:  logistic , solver:  lbfgs , batch_size:  15 }

ROC AUC SCORE:  0.7389241629315004 


              precision    recall  f1-score   support

           0       0.88      0.66      0.75      2877
           1       0.41      0.73      0.53       955

    accuracy                           0.67      3832
   macro avg       0.65      0.69      0.64      3832
weighted avg       0.76      0.67      0.70      3832
 






/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


{hidden_layer_sizes:  128 , activation:  logistic , solver:  lbfgs , batch_size:  20 }

ROC AUC SCORE:  0.74246224342911 


              precision    recall  f1-score   support

           0       0.87      0.65      0.75      2877
           1       0.41      0.72      0.52       955

    accuracy                           0.67      3832
   macro avg       0.64      0.68      0.63      3832
weighted avg       0.76      0.67      0.69      3832
 






/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


{hidden_layer_sizes:  (16, 32, 64) , activation:  relu , solver:  lbfgs , batch_size:  15 }

ROC AUC SCORE:  0.7548300567599685 


              precision    recall  f1-score   support

           0       0.88      0.67      0.76      2877
           1       0.42      0.72      0.53       955

    accuracy                           0.68      3832
   macro avg       0.65      0.69      0.64      3832
weighted avg       0.76      0.68      0.70      3832
 






/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


{hidden_layer_sizes:  (16, 32, 64) , activation:  relu , solver:  lbfgs , batch_size:  20 }

ROC AUC SCORE:  0.7489593399174169 


              precision    recall  f1-score   support

           0       0.87      0.67      0.76      2877
           1       0.41      0.71      0.52       955

    accuracy                           0.68      3832
   macro avg       0.64      0.69      0.64      3832
weighted avg       0.76      0.68      0.70      3832
 






KeyboardInterrupt: ignored